In [0]:
import requests
from pyspark.sql.types import *
from pyspark.sql.functions import col, coalesce

api_url = "https://api.restful-api.dev/objects"
response = requests.get(api_url)
data = response.json()

schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("data", StructType([
        StructField("color_lower", StringType(), True),  
        StructField("capacity", StringType(), True),
        StructField("capacity_gb", StringType(), True),   
        StructField("price", DoubleType(), True),
        StructField("generation", StringType(), True),
        StructField("year", IntegerType(), True),
        StructField("cpu_model", StringType(), True),     
        StructField("hard_disk_size", StringType(), True),
        StructField("strap_colour", StringType(), True),  
        StructField("case_size", StringType(), True),     
        StructField("color_upper", StringType(), True),   
        StructField("description", StringType(), True),
        StructField("capacity_alt", StringType(), True),  
        StructField("screen_size", DoubleType(), True),   
        StructField("generation_alt", StringType(), True),
        StructField("price_alt", StringType(), True)      
    ]), True)
])


df = spark.createDataFrame(data, schema=schema)

flat_df = df.select(
    "id",
    "name",
    "data.*"
).withColumn("final_color", coalesce(col("color_lower"), col("color_upper")))

flat_df.write.format("delta").mode("overwrite").saveAsTable("flattened_api_objects")

display(spark.table("flattened_api_objects"))
